In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard library imports.
import itertools

# Related third party imports.
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation
import src.data.graph_utilities as gu

In [6]:
network_folder = '../data/processed/networks'
network_name = '25_italy'
factor = 0.5

# Load data
g = nx.read_gml(f'{network_folder}/{network_name}.gml', label="id")
diameter = gu.get_graph_diameter(g)
ml = diameter * factor

ns = NetworkSimulation(network_name, **{'max_length':ml, 'shortest_k':16})
ns.generate_new_vSDN_requests(number_of_requests=20, size_of_requests=15, TTL_range=2)

S, H, C = list(g.nodes), list(g.nodes), list(g.nodes)
H_pairs = list(itertools.combinations_with_replacement(H,2))
HS_pairs = list(itertools.product(H, S))
HHS_pairs = list(itertools.product(H_pairs, S))
CS_pairs = list(itertools.product(C, S))
R = {r.get_id():r.get_switches() for r in ns.new_vSDN_requests}
CR_pairs = list(itertools.product(C, R.keys()))

allowed_switch_H_pairs = gu.get_allowed_hypervisor_pairs_by_switch(ns.network_operator.triplets_by_switches)
allowed_cs_H_pairs = ns.network_operator.quartets_by_cs

In [8]:
model = gp.Model("mip1")

active_hypervisors = model.addVars(H, vtype=GRB.BINARY)
hypervisor_controls_switch = model.addVars(HS_pairs, vtype=GRB.BINARY)
hypervisor_pair_controls_switch = model.addVars(HHS_pairs, vtype=GRB.BINARY)
controller_controls_switch = model.addVars(CS_pairs, vtype=GRB.BINARY)
controller_controls_request = model.addVars(CR_pairs, vtype=GRB.BINARY)
controllable_request = model.addVars(R.keys(), vtype=GRB.BINARY)

# Only active hypervisors can control switches, Hypervisors without controlled switches are inactive
c_1 = model.addConstrs(active_hypervisors[h] == gp.or_([hypervisor_controls_switch[(h,s)] for s in S]) for h in H)

# Each switch is controlled by a pair of hypervisors except when there is a hypervisor at the switch’s location
c_2a = model.addConstrs(active_hypervisors[s] <= hypervisor_controls_switch[(s,s)] for s in S)
# c_2a = model.addConstrs(active_hypervisors[s] <= hypervisor_pair_controls_switch[((s,s),s)] for s in S)
c_2b = model.addConstrs(hypervisor_controls_switch[(s,s)] + gp.LinExpr([(1, hypervisor_controls_switch[(h,s)]) for h in H]) == 2 for s in S)
# c_2b = model.addConstrs(gp.quicksum([hypervisor_pair_controls_switch[((h1,h2),s)] for h1,h2 in H_pairs]) == 1 for s in S)

# The hypervisor pair controls the switch if both of them are controlling it
c_3 = model.addConstrs(hypervisor_pair_controls_switch[((h1,h2),s)] == gp.and_(hypervisor_controls_switch[(h1,s)], hypervisor_controls_switch[(h2,s)]) for (h1,h2),s in HHS_pairs if not (h1==h2 and h1!=s))

# Only valid triplets (T) can appear
c_4 = model.addConstrs(gp.quicksum([hypervisor_pair_controls_switch[((h1,h2),s)] for h1,h2 in allowed_switch_H_pairs[s]]) == 1 for s in S)
# c_5a = model.addConstrs(gp.quicksum([hypervisor_pair_controls_switch[((h1,h2),s)] for h1,h2 in not_allowed_switch_H_pairs[s]]) == 0 for s in S)
# c_5b = model.addConstrs(hypervisor_pair_controls_switch[((h1,h2),s)] == 0 for h1,h2 in not_allowed_switch_H_pairs[s] for s in S)
# c_5c = model.addConstrs(hypervisor_controls_switch[(h1,s)] + hypervisor_controls_switch[(h2,s)] <= 1 for h1,h2 in not_allowed_switch_H_pairs[s] for s in S)

# If a good hypervisor pair controls the switch then the controller is able to control it
c_6 = model.addConstrs(controller_controls_switch[(c,s)] == gp.or_([hypervisor_pair_controls_switch[((h1,h2),s)] for h1,h2 in allowed_cs_H_pairs.get((c,s), [])]) for c,s in CS_pairs)

# The request can be accepted with the controller if it can control all of its switches
c_7 = model.addConstrs(controller_controls_request[(c,r)] == gp.and_([controller_controls_switch[(c,s)] for s in R[r]]) for c,r in CR_pairs)

# The request is acceptable if there is a controller that can control all of its switches
c_8 = model.addConstrs(controllable_request[r] == gp.or_([controller_controls_request[(c,r)] for c in C]) for r in R)

c_9 = model.addConstr(gp.quicksum(active_hypervisors) <= 8)

# Minimize the number of hypervisors
model.setObjective(gp.quicksum(controllable_request)/len(R), GRB.MAXIMIZE)
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 76 rows, 9920 columns and 2130 nonzeros
Model fingerprint: 0x86a02955
Model has 8695 general constraints
Variable types: 0 continuous, 9920 integer (9920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e-02, 5e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve added 3312 rows and 0 columns
Presolve removed 0 rows and 7672 columns
Presolve time: 0.25s
Presolved: 3388 rows, 2248 columns, 18499 nonzeros
Variable types: 0 continuous, 2248 integer (2248 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 7.000000e-01, 1718 iterations, 0.23 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.70000    0  201   -0.0000

In [9]:
for h,v in active_hypervisors.items():
    if v.x > 0.9:
        print('%s' % (h))
print('Obj: %g' % model.objVal)

0
4
5
13
19
20
21
23
Obj: 0.45


In [10]:
for ((h1,h2),s),v in hypervisor_pair_controls_switch.items():
    if v.x > 0.9:
        print('%2s %2s %2s' % (s, h1, h2))

 0  0  0
 1  0 20
 3  0 20
 6  0 20
 7  0 20
 8  0 20
 9  0 20
10  0 20
11  0 20
12  0 20
15  0 20
16  0 20
17  0 20
19  0 20
22  0 20
 2  0 24
18  0 24
21  0 24
 4  4  4
 5  5  5
13 13 13
14 14 14
20 20 20
23 23 23
24 24 24


In [10]:
for r,v in controllable_request.items():
    if v.x > 0.5:
        print(R[r])

[0, 6, 7, 9, 10, 12, 15, 16, 17, 18, 19, 20, 21, 22, 24]
[1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 19, 20, 21]
[0, 3, 4, 5, 6, 7, 9, 12, 15, 16, 19, 20, 21, 22, 24]
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 18, 19, 20, 21]
[1, 2, 6, 7, 8, 9, 10, 11, 15, 16, 17, 19, 20, 21, 22]
[0, 1, 2, 6, 8, 9, 12, 16, 17, 18, 19, 20, 21, 22, 23]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 19, 20, 21, 22]
[0, 1, 3, 6, 7, 8, 9, 10, 11, 15, 16, 17, 18, 20, 24]
[0, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 18, 24]
